In [1]:
pip install "camelot-py[cv]" numpy pandas tabula-py ghostscript

Note: you may need to restart the kernel to use updated packages.


In [4]:
from ctypes.util import find_library
find_library("/usr/local/lib/docker/")

In [3]:
import tkinter

In [5]:
import ghostscript

RuntimeError: Can not find Ghostscript library (libgs)